# Time Domain Analysis: Simulations of Transients with Roman I-Sim

***

## Kernel Information and Read-Only Status

To run this notebook, please select the "Roman Calibration" kernel at the top right of your window.

This notebook is read-only. You can run cells and make edits, but you must save changes to a different location. We recommend saving the notebook within your home directory, or to a new folder within your home (e.g. <span style="font-variant:small-caps;">file > save notebook as > my-nbs/nb.ipynb</span>). Note that a directory must exist before you attempt to add a notebook to it.

## IMPORTANT: Astrocut Version

Be sure to upgrade the astrocut version to 1.0.1 before running this tutorial. Check it now:

In [ ]:
import astrocut

print(f'astrocut version = {astrocut.__version__}')

**If you have astrocut version 1.0.0**, then uncomment the following cell, run it, and then restart the kernel. You can restart the kernel by going to the menubar, selecting "Kernel" then "Restart Kernel...". You can also press the button at the top of the notebook that looks like a circle with an arrow.

**Failure to upgrade astrocut to version 1.0.1 will result in errors in the tutorial notebook cells.**

In [ ]:
#!pip install --upgrade astrocut==1.0.1

## Imports

**TODO: Populate this list.**

In [ ]:
pip install sncosmo

In [ ]:
# Supernova model imports
import sncosmo

# General imports
import argparse
from astropy.coordinates import SkyCoord
from astropy.nddata import Cutout2D
from astropy.table import Table, vstack
from astropy.time import Time
import astropy.units as u
from astropy.utils.exceptions import AstropyDeprecationWarning
from astropy.wcs import WCS
import copy
from dataclasses import dataclass
import glob
import numpy as np
import shutil
import os
from typing import Union

# Plotting imports
import matplotlib.pyplot as plt
from matplotlib import animation
from astropy.visualization import simple_norm

# Roman software imports
import asdf
from astrocut import ASDFCutout
import galsim
import pysiaf
import roman_datamodels as rdm
from romancal.pipeline import ExposurePipeline
from romanisim import gaia, bandpass, catalog, log, wcs, persistence, parameters, ris_make_utils as ris
from romanisim.image import inject_sources_into_l2

# Warning handling
import warnings

# Ignore warnings from the 'astropy.wcs' module
# When creating cutouts -> WARNING: Polynomial distortion is not implemented.
warnings.filterwarnings('ignore', module='astropy.wcs')

# Ignore AstropyDeprecationWarning
# When creating cutouts -> WARNING: AstropyDeprecationWarning: The class "Fits" has been renamed to "FITS" in version 7.0. 
# The old name is deprecated and may be removed in a future version. Use FITS instead. [astropy.units.format]
warnings.filterwarnings('ignore', category=AstropyDeprecationWarning)

## Introduction

In this tutorial, we will explore how to create a series of simulations of a variable source using the Roman WFI image simulator Roman I-Sim. We will use a transient source, in this case a Type Ia supernova, for our example, but the methods described here can be extended to any type of variable source for which we have a light curve.

A light curve is a representation of the brightness of a source as a function of time. In this tutorial, we will use a Python package (`sncosmo`) to generate a model light curve for us, but there are numerous resources available for model and template light curves for both transient and cyclic variables of various types.

## Simulating Transients with Roman I-Sim

We have three options when we want to simulate time-domain observations, and while all are equally viable choices, there are some reasons why you may want to choose one over the other. These options are:

1. Simulate a Level 1 (L1; uncalibrated ramp cube) file for each observation epoch with a catalog of all (both varying and non-varying) sources unique to each epoch, and calibrate each L1 file to Level 2 (L2; calibrated rate image) using the [Exposure Pipeline](../exposure_pipeline/exposure_pipeline.ipynb).
2. Simulate a L1 file for each observation epoch with a catalog of non-varying sources, calibrate each L1 file to L2 using the Exposure Pipeline, and use [Roman I-Sim](../romanisim/romanisim.ipynb) to inject variable sources into each L2 image.
3. Simulate a single L1 observation with a catalog of non-varying sources, calibrate it to L2 using the Exposure Pipeline, and use Roman I-Sim to inject variable sources into copies of the L2 image for each observation epoch.

At present, Roman I-Sim does not support source injection into the L1 file, but this can be accomplished by updating the input source catalog for each observation epoch. This is method 1 above.

Both methods 1 and 2 result in a large volume of data on disk and significant processing time, but yield the most realistic results, allowing for movement of the sources in the focal plane between observations and different realizations of the noise properties. Method 3 is the most computationally efficient method, but results in images where the sky background and non-varying sources are identical in every image.

In this tutorial, we will adopt method 3, but provide tips for how to implement the other two methods. The [Roman I-Sim](../romanisim/romanisim.ipynb) and [Exposure Pipeline](../exposure_pipeline/exposure_pipeline.ipynb) tutorials provide background information and examples to implement methods 1 and 2.

### Catalog Setup (Host Galaxy)

We need to begin by making some decisions about the non-variable source catalog. First, we need to know what part of the sky we will be observing, as that helps us to place the sources on the WFI detector(s) that we will simulate. In this example, we put our supernova host galaxy at the center of detector WFI01 with the WFI pointed at (RA, Dec) = (72.5, -30.7) degrees and a WFI position angle of -60 degrees (telescope roll angle of 0 degrees).

To start, we copy the `PointWFI` class we created and described in the [Roman I-Sim](../romanisim/romanisim.ipynb) tutorial. In this tutorial, we have added another method (`get_detector_coords()`) to the class that returns the RA and Dec for a science pixel position on a detector for a given WFI pointing.

In [ ]:
@dataclass(init=True, repr=True)
class PointWFI:
    """
    Inputs
    ------
    ra (float): Right ascension of the target placed at the geometric 
                center of the Wide Field Instrument (WFI) focal plane
                array. This has units of degrees.
    dec (float): Declination of the target placed at the geometric
                 center of the WFI focal plane array. This has units
                 of degrees.
    position_angle (float): Position angle of the WFI relative to the V3 axis
                            measured from North to East. A value of 0.0 degrees
                            would place the WFI in the "smiley face" orientation
                            (U-shaped) on the celestial sphere. To place WFI
                            such that the position angle of the V3 axis is 
                            zero degrees, use a WFI position angle of -60 degrees.

    Description
    -----------
    To use this class, instantiate it with your initial pointing like so:

        >>> point = PointWFI(ra=30, dec=-45, position_angle=10)
    
    and then dither using the dither method:

        >>> point.dither(x_offset=10, y_offset=140)

    This would shift the WFI 10 arcseconds along the X-axis of the WFI
    and 140 arcseconds along the Y-axis of the WFI. These axes are in the ideal
    coordinate system of the WFI, i.e, with the WFI oriented in a U-shape with 
    +x to the right and +y up. You can pull the new pointing info out of the object 
    either as attributes or by just printing the object:

        >>> print(point.ra, point.dec)
        >>> 29.95536280064078 -44.977122003232786

    or

        >>> point
        >>> PointWFI(ra=29.95536280064078, dec=-44.977122003232786, position_angle=10)
    """

    # Set default pointing parameters
    ra: float = 0.0
    dec: float = 0.0
    position_angle: float = 60.0

    # Post init method sets some other defaults and initializes
    # the attitude matrix using PySIAF.
    def __post_init__(self) -> None:
        self.siaf_aperture = pysiaf.Siaf('Roman')['WFI_CEN']
        self.v2_ref = self.siaf_aperture.V2Ref
        self.v3_ref = self.siaf_aperture.V3Ref
        self.attitude_matrix = pysiaf.utils.rotations.attitude(self.v2_ref, self.v3_ref, self.ra,
                                        self.dec, self.position_angle)
        self.siaf_aperture.set_attitude_matrix(self.attitude_matrix)

        # Compute the V3 position angle
        self.tel_roll = pysiaf.utils.rotations.posangle(self.attitude_matrix, 0, 0)

        # Save initial pointing
        self.att0 = self.attitude_matrix.copy()

        # Save a copy of the input RA and Dec in case someone needs it
        self.ra0 = copy.copy(self.ra)
        self.dec0 = copy.copy(self.dec)

    def dither(self, x_offset: Union[int, float],
               y_offset: Union[int, float]) -> None:
        """
        Purpose
        -------
        Take in an ideal X and Y offset in arcseconds and shift the telescope
        pointing to that position.

        Inputs
        ------
        x_offset (float): The offset in arcseconds in the ideal X direction.

        y_offset (float): The offset in arcseconds in the ideal Y direction.
        """

        self.ra, self.dec = self.siaf_aperture.idl_to_sky(x_offset, y_offset)

    def get_detector_coords(self, detector: int, x: float, y: float) -> tuple:
        """
        Purpose
        -------
        Convert a pixel position on a WFI detector to RA and Dec using the 
        pointing information of the WFI center.

        Inputs
        ------
        detector (integer): WFI detector number 1 - 18.

        x (float): Science X zero-indexed pixel position.

        y (float): Science Y zero-indexed pixel position.
        """

        det_siaf = pysiaf.Siaf('Roman')[f'WFI{detector:02d}_FULL']
        det_siaf.set_attitude_matrix(self.attitude_matrix)
        det_ra, det_dec = det_siaf.sci_to_sky(x + 1, y + 1)
        
        return det_ra, det_dec

Now that we have our `PointWFI` class, we can use it to determine the position of our supernova host galaxy at the center of WFI01:

In [ ]:
pointing = PointWFI(ra=72.5, dec=-30.7)
ra, dec = pointing.get_detector_coords(1, 2043.5, 2043.5)

With the galaxy RA and Dec, we can create a basic Roman I-Sim catalog that contains a single galaxy. For this example, we will set the host galaxy to have a spectral energy distribution (SED) that is flat in frequency. This is, of course, unrealistic, but is simple for our example. This can be changed below by altering the values for "F129" and "F213" in the dictionary. Also note that we are only setting the F129 and F213 fluxes for the simulation, but this can also be expanded. We set the host galaxy to be 15 AB magnitudes in brightness and choose values for the other galaxy parameters that Roman I-Sim needs, i.e., Sérsic index, half-light radius, position angle, and minor-to-major axis ratio.

For more information on Roman I-Sim catalogs, please see the [Roman I-Sim tutorial](../romanisim/romanisim.ipynb) and the [Roman I-Sim documentation](https://romanisim.readthedocs.io/en/latest/).

In [ ]:
m_gal_ab = 15  # AB magnitude of host galaxy

galaxy_cat = Table({'ra': [ra],
                    'dec': [dec],
                    'type': ['SER'],
                    'n': [1.5],
                    'half_light_radius': [6.0],
                    'pa': [30.0],
                    'ba': [0.3],
                    'F129': [10**(-0.4 * m_gal_ab)],
                    'F213': [10**(-0.4 * m_gal_ab)]})

### Simulating the L1 Image and Running the Exposure Pipeline

As we are simulating a single L1 file and injecting sources at L2 for our different epochs, we can now simulate the L1 file. Recall from the [Roman I-Sim](../romanisim/romanisim.ipynb) tutorial, we have a function that we can call to run the simulation. We have modified it slightly to add the `position_angle` optional argument for the WFI position angle (in degrees), which we pass to the `fill_in_parameters()` method argument `pa_aper` below. For our example, this is set to -60 degrees as that is how we set up the pointing above.

In [ ]:
def run_romanisim(catalog, ra=80.0, dec=30.0, obs_date = '2026-10-31T00:00:00', sca=1, expnum=1, optical_element='F106', 
                  ma_table_number=3, level=2, filename='r0003201001001001004', seed=5346, position_angle=0.0):

    cal_level = 'cal' if level == 2 else 'uncal'
    filename = f'{filename}_{expnum:04d}_wfi{sca:02d}_{optical_element.lower()}_{cal_level}.asdf'

    # Set other arguments for use in Roman I-Sim. The code expects a specific format for these, so this 
    # is a little complicated looking.
    parser = argparse.ArgumentParser()
    parser.set_defaults(usecrds=True, stpsf=True, level=level, filename=filename, drop_extra_dq=True)
    args = parser.parse_args([])

    # Set reference files to None for CRDS
    for k in parameters.reference_data:
        parameters.reference_data[k] = None

    # Set Galsim RNG object
    rng = galsim.UniformDeviate(seed)

    # Set default persistance information
    persist = persistence.Persistence()

    # Set metadata
    metadata = ris.set_metadata(date=obs_date, bandpass=optical_element, sca=sca, ma_table_number=ma_table_number)

    # Update the WCS info
    wcs.fill_in_parameters(metadata, SkyCoord(ra, dec, unit='deg', frame='icrs'), boresight=False, 
                           pa_aper=position_angle)

    # Run the simulation
    sim_result = ris.simulate_image_file(args, metadata, catalog, rng, persist)

    # Clean up the memory
    del sim_result

Let's run that now with our `galaxy_cat` catalog and generate a L1 image. In the cells below, we have commented out the lines that would generate the simulations for the F213 data. The final cutout files that we make at the end of the simulation will be available for both F129 and F213 in the Nexus S3 bucket.

In [ ]:
# Run WFI01 with F129 filter
run_romanisim(galaxy_cat, ra=pointing.ra, dec=pointing.dec, optical_element='F129', level=1, 
              filename='r9999901001001001001')

## Run WFI01 with F213 filter
#run_romanisim(galaxy_cat, ra=pointing.ra, dec=pointing.dec, optical_element='F213', level=1, 
#              filename='r9999902002002002002')

And now we can process both L1 files through the Exposure Pipeline. Since we have no stars in our example image, we will skip the source catalog and tweakreg steps. For more information on these steps, see the [Exposure Pipeline](../exposure_pipeline/exposure_pipeline.ipynb) tutorial.

In [ ]:
_ = ExposurePipeline.call('r9999901001001001001_0001_wfi01_f129_uncal.asdf', save_results=True,
                          steps={'source_catalog': {'skip': True},
                                 'tweakreg': {'skip': True}})

#_ = ExposurePipeline.call('r9999902002002002002_0001_wfi01_f213_uncal.asdf', save_results=True,
#                          steps={'source_catalog': {'skip': True},
#                                 'tweakreg': {'skip': True}})

Next, we read in the files we just made:

In [ ]:
dm_f129 = rdm.open('r9999901001001001001_0001_wfi01_f129_cal.asdf')
# dm_f213 = rdm.open('r9999902002002002002_0001_wfi01_f213_cal.asdf')

Let's take a look at one of our images and verify that our galaxy looks like what we expect:

In [ ]:
# Display the area around the galaxy in the F129 L2 image
# centered at (X, Y) = (2043.5, 2043.5).
fig, ax = plt.subplots()
arange = [1900, 2200]
norm = simple_norm(dm_f129.data, 'asinh', vmin=0.5, vmax=3)
ax.imshow(dm_f129.data[arange[0]:arange[1], arange[0]:arange[1]], norm=norm, origin='lower',
         extent=[arange[0], arange[1], arange[0], arange[1]], cmap='afmhot')

ax.set_xlabel('Science X (pixels)')
ax.set_ylabel('Science Y (pixels)');

This looks great! Our galaxy is nicely centered where we expect it to be. Next, we'll look at how to generate light curves for our supernova and inject it into the image.

### Simulating a Type Ia Supernova

Now that we have our host galaxy parameters, we can set up our supernova model. In this tutorial, we use `sncosmo` to generate a lightcurve given several input model criteria. In particular, we choose the SALT3-NIR model ([Pierel et al. 2022](https://ui.adsabs.harvard.edu/abs/2022ApJ...939...11P/abstract)) for this tutorial. There are additional options available in `sncosmo`. Please see the [sncosmo documentation](https://sncosmo.readthedocs.io/en/stable/) for more information. 

For our example supernova, we set the redshift to $z=1.2$, and normalize the peak brightness to 20 AB magnitudes in the F158 filter. We also have to choose a date for t0, the meaning of which varies from model to model. In this case, it corresponds to maximum flux. We will set t0 below to be 2027-06-08 13:25:42. Finally, we choose a sampling of observation dates around the peak (from -40 days to +115 days in 5-day increments). The SALT3-NIR model also requires two more parameters ("x1" and "c"), which are just scaling parameters. We set these to 1 in the example below, and refer the reader to Pierel et al. (2022) for more information on the model.

In [ ]:
# Choose the supernova RA and Dec
sn_ra, sn_dec = pointing.get_detector_coords(1, 2000, 2000)

sn_model = sncosmo.Model(source='salt3-nir')
wfi_filters = ['F129', 'F213']

t0 = Time('2027-06-08T13:25:42').mjd
delta_times = np.arange(-40, 115, 5)
obs_dates = delta_times + t0

# Set properties of the supernova
# z = redshift
# t0 = peak date in MJD
# x1 and c are scaling parameters of the SALT model and can be set to 1
sn_properties = {'z': 1.2,
                't0': t0,
                 'x1': 1,
                 'c': 1}
sn_model.set(**sn_properties)

peak_mag = 20
sn_model.set_source_peakmag(peak_mag, 'f158', 'ab')

With the model set up, we can now generate AB magnitudes for our supernova using the `.bandmag()` method on the `sn_model` variable. We also need to convert the AB magnitudes to units of maggies for input into Roman I-Sim. Recall that a maggie is a linearized flux unit normalized to the AB magnitude zeropoint, i.e.:

$\mathrm{maggies} = 10^{-0.4 m_{\mathrm{AB}}}$

In the next cell, we will generate the AB magnitudes from the model and convert them to maggies.

In [ ]:
# Now get the AB magnitudes in our bandpasses and convert
# those to maggies for Roman I-Sim
maggies = {}
for wf in wfi_filters:
    tmp = sn_model.bandmag(wf.lower(), 'ab', obs_dates)
    maggies[wf] = 10**(-0.4 * tmp)

maggies

Now we have brightnesses in AB magnitudes and maggies for all of our observation epochs. Let's take a quick look at a light curve now for our two filters:

In [ ]:
fig, ax = plt.subplots()
ax.plot(obs_dates, maggies['F129'], linestyle='-', marker='o', color='orange', label='F129')
ax.plot(obs_dates, maggies['F213'], linestyle='-', marker='o', color='red', label='F213')
ax.set_ylabel('Flux (maggies)')
ax.set_xlabel ('MJD (observed frame)')
yrange = ax.get_ylim()
ax.vlines(x=[t0], ymin=-1, ymax=1, label='Peak Time', zorder=-100)
ax.set_ylim(yrange)
ax.legend();

This looks exactly what we might expect. In the near-infrared, it is common to see a secondary peak 30-40 days after the peak brightness of a type Ia supernova. We observe a strong secondary peak in the F213 light curve. We also see that the F213 light curve peaks slightly before both the F129 filter and the peak date. The supernova also appear to generall be brighter in the F213 filter.

Now we will inject our light curve into copies of our L2 image.

### Injecting the Supernova into the Image

For each observation epoch, we will make a copy of the L2 file (for both filters), and inject the supernova. We do this using Roman I-Sim and by making a catalog of just the source(s) we wish to inject. See the [Roman I-Sim](../romanisim/romanisim.ipynb) tutorial for more information. To simplify this, let's make a function:

In [ ]:
# Define a function to create the catalog for sources
# that we want to inject into our L2 image.

def make_cat(ra, dec, fluxes, filename='injection_cat.ecsv'):

    filtlist = 'F062 F087 F106 F129 F146 F158 F184 F213'.split()
    tab = Table([ra, dec], names = ('ra', 'dec'))
    src_types = np.array(['PSF' for i in ra])
    tab['type'] = src_types
    for k, v in fluxes.items():
        if k.upper() in filtlist:
            tab[k.upper()] = v
        else:
            raise ValueError(f'Incorrect filter name {k}!')

    return tab

Let's do the source injection once to test it, and then we can loop over it for all of our epochs in both filters. Let's pick an epoch several days into the light curve to make sure the source is easily visible. We also need to pick the right ascension and declination of our supernova now. We will place it close to the host galaxy at (X, Y) = (2000, 2000). Now that we have our L2 images, we can use the WCS in them to determine the RA and Dec at that location:

In [ ]:
sn_ra, sn_dec = f129_l2.meta.wcs(2000, 2000)
print(f'Supernova (RA, Dec) = ({sn_ra:.5f}, {sn_dec:.5f})')

Now let's make the catalog for the supernova at time index 8, which is (5 days) x 8 = 40 days from the start of the light curve. The light curve began at t0 - 40 days, so this corresponds to the peak brightness.

In [ ]:
epoch = 8
fluxes = {}
for key, value in maggies.items():
    fluxes[key] = value[epoch]

catalog = make_cat([sn_ra], [sn_dec], fluxes)
catalog

When we simulated our L1 images, we overrode the parameters that Roman I-Sim uses by default and forced Roman I-Sim to use CRDS to get calibration reference files for the simulation. For source injection, Roman I-Sim currently only uses values in its library. These can be changed, but we set them all to `None` in a previous cell. In the next cell, we use Python's `importlib` to reload the `romanisim.parameters` module to get the reset the defaults.

In [ ]:
import importlib
importlib.reload(parameters)

Next, we inject our supernova into our image:

In [ ]:
result = inject_sources_into_l2(dm_f129.copy(), catalog, stpsf=True)

Now let's plot the original image and the injected image side-by-side for comparison:

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2)
norm = simple_norm(dm_f129.data, 'asinh', vmin=0.5, vmax=3)
arange = [1900, 2200]

axs[0].imshow(dm_f129.data[arange[0]:arange[1], arange[0]:arange[1]], norm=norm, 
              cmap='afmhot', origin='lower',
              extent=[arange[0], arange[1], arange[0], arange[1]])

axs[0].set_title('Original')

axs[1].imshow(result.data[arange[0]:arange[1], arange[0]:arange[1]], norm=norm, 
              cmap='afmhot', origin='lower',
              extent=[arange[0], arange[1], arange[0], arange[1]])

axs[1].set_title('Injected')

for ax in axs:
    ax.set_xlabel('Science X (pixels)')
    ax.set_ylabel('Science Y (pixels)')

plt.tight_layout();

We can very clearly see a new star where there was not one before, and it is centered exactly where we expect at (X, Y) = (2000, 2000).

For plotting purposes, we have only been showing the range (X, Y) = (1900 - 2200, 1900 - 2200). But this is in a way a "cheat" because these arrays are much larger. Since we only care about such a small region of 400 x 400 pixels, we can use `astrocut` to make cutouts around this region and save those for later analysis. In the next cell, we will put all of this together by looping over each observation epoch and injecting the supernova into the L2 image, and also using `astrocut` to make a cutout of each epoch.

**Note:** The current version of `astrocut` requires the input to be an ASDF file on disk, so we will write out our files to disk as we do this and then clean them up after. **This is not recommended behavior.** A future version of `astrocut` will allow for working on files in memory.

**Note:** the following cell will take several minutes to run as it loops over all of the epochs in our time-domain simulation.

In [ ]:
# Cutout information
cutout_size = 400  # pixels
cutout_ra, cutout_dec = dm_f129.meta.wcs(2050, 2050)
cutout_coord = SkyCoord(ra=cutout_ra * u.deg, dec=cutout_dec * u.deg)

# Loop over epochs
for i, od in enumerate(obs_dates):
    fluxes = {}
    for key, value in maggies.items():
        fluxes[key] = value[i]

    # Make the catalog of the supernova for this date
    catalog = make_cat([sn_ra], [sn_dec], fluxes)

    # Inject source into images
    result_f129 = inject_sources_into_l2(dm_f129.copy(), catalog, stpsf=True)
    #result_f213 = inject_sources_into_l2(dm_f213.copy(), catalog, stpsf=True)

    # Save the file temporarily.
    # This is BAD. This will be removed when astrocut is updated to work in memory.
    day = int(od)
    result_f129.save(f'f129_{day}.asdf')
    #result_f213.save(f'f213_{day}.asdf')

    # Make the cutout.
    cutout_f129 = astrocut.ASDFCutout(f'f129_{day}.asdf', cutout_coord, cutout_size)
    #cutout_f213 = astrocut.ASDFCutout(f'f213_{day}.asdf', cutout_coord, cutout_size)

    # Update the metadata of the cutout with some things we want to keep and then save to disk.
    # First F129...
    asdf_f129 = cutout_f129.asdf_cutouts[0]
    asdf_f129['roman']['meta']['photometry'] = dict(dm_f129.meta.photometry)
    asdf_f129['roman']['meta']['exposure'] = {'start_time': od}
    asdf_f129.write_to(f'roman_sn1a_{day}_f129.asdf')
    os.remove(f'f129_{day}.asdf')

    ## Next F213...
    #asdf_f213 = cutout_f213.asdf_cutouts[0]
    #asdf_f213['roman']['meta']['photometry'] = dict(dm_f213.meta.photometry)
    #asdf_f213['roman']['meta']['exposure'] = {'start_time': od}
    #asdf_f213.write_to(f'roman_sn1a_{day}_f213.asdf')
    #os.remove(f'f213_{day}.asdf')

Now we have cutouts with our supernova injected at each observation epoch. In the next tutorial, we will show how to perform photometry on our supernova and fit the light curve with a model. For now though, let's do something fun and make an animated gif of the star we just exploded.

To make an animated gif, we will use the `matplotlib.animation` module. 

In [ ]:
files = glob.glob('roman_sn1a_*_f129.asdf')
files.sort()

fig, ax = plt.subplots()

imgs = []
for i, f in enumerate(files):
    af = asdf.open(f)
    if i == 0:
        norm = simple_norm(af['roman']['data'], 'asinh', vmin=0.5, vmax=3)
        ax.imshow(af['roman']['data'], origin='lower', cmap='afmhot', norm=norm)  # Show initial image
    imgs.append([ax.imshow(af['roman']['data'], origin='lower', cmap='afmhot', norm=norm, animated=True)])

ani = animation.ArtistAnimation(fig, imgs, interval=200, blit=True, repeat_delay=1000)
ani.save('sn1a.gif', writer=animation.PillowWriter(fps=15))

Next, we can load the gif in the notebook (or you can download it, too!):

![SN1a](sn1a.gif)

## Additional Resources


## About This Notebook

**Author:** Melissa Shahbandeh, Tyler Desjardins\
**Updated On:** 2025-06-07

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 